In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pyfolio as pf
import pandas as pd
import numpy as np
# from taa_lib import taa
# from baa_lib import baa
# from dualmom_lib import dualmom
# from meta_lib import meta
from leveraged_ew_meta_lib import leveraged_ew_meta
import common_perf_ana as cpa
import datetime

In [ ]:

meta_parameters = {
    'meta_rebalance_unit' : 'Month',
    'meta_rebalance_freq' : 1,
    'meta_rebalance_shift' : 0,
    'meta_lb_period' : 3,
    'meta_skipped_period' : 0,
    'meta_fixed_substrat_weights' : {'DoubleMom' : 3, 'BAA_AggDef' : 2, 'BAA_BalDef' : 2, 'TAA' : 1},
    'meta_rel_mom_substrat_weights' : [1, 2, 4, 8], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf.]
    'meta_sharpe_substrat_weights' : [1, 2, 4, 8], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf.]
    'meta_sortino_substrat_weights' : [1, 2, 4, 8], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf.]
    'meta_rel_mom_abs_threshold' : -1,
    'meta_sharpe_abs_threshold' : -100,
    'meta_sortino_abs_threshold' : -100,
    'meta_start_date' : '2004-01-01',
    'meta_end_date' : datetime.date.today()
}

meta_leverage_parameters = {
    # 'meta_overall_leverage' : 1,
    # 'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    # 'meta_ETF_perf_leverage' : [1, 1, 1, 1, 1],
    # 'meta_monthly_seas_based_leverage' : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # 'meta_leverage_lookback_years': 5,
    # 'meta_tlt_based_leverage' : [1, 1, 1, 1] #[Bull, Rebound, Correction, Bear]
    'meta_overall_leverage' : 1,
    'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    'meta_ETF_perf_leverage' : [2, 1.5, 1, 0.75, 0.5],
    'meta_monthly_seas_based_leverage' : [1.5, 1.5, 1.25, 1.25, 1, 1, 1, 1, 1, 1, 1, 1],
    'meta_leverage_lookback_years': 5,
    'meta_tlt_based_leverage' : [1.1, 1.1, 1, 0.8] #[Bull, Rebound, Correction, Bear]
}

taa_parameters = {
    'taa_ticker_list' : ['VNQ', 'EEM', 'DBC', 'SPY', 'QQQ', 'TLT'],
    'taa_perc_ch_lb_list' : [60, 120, 180, 250],
    'taa_vol_lb' : 20,
    'taa_perc_ch_up_thres' : 0.75,
    'taa_perc_ch_low_thres' : 0.25,
    'taa_rebalance_unit' : 'Month',
    'taa_rebalance_freq' : 1,
    'taa_rebalance_shift' : 0
}

bold_parameters = {
    'baa_ticker_list_canary' : ['SPY', 'EEM', 'EFA', 'AGG'],
    'baa_ticker_list_defensive' : ['TIP', 'DBC', 'BIL', 'IEF', 'TLT', 'LQD', 'AGG'],
    # 'baa_ticker_list_aggressive' : ['QQQ'],
    'baa_ticker_list_aggressive' : ['QQQ', 'EEM', 'EFA', 'AGG'],
    'baa_ticker_list_balanced' : ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'VNQ', 'DBC', 'GLD', 'TLT', 'HYG', 'LQD'],
    'baa_rebalance_unit' : 'Month',
    'baa_rebalance_freq' : 1,
    'baa_rebalance_shift' : 0,
    'baa_skipped_period' : 0,
    'baa_no_played_ETFs' : {'offAgg' : 1, 'offBal' : 6, 'def' : 3},
    'baa_abs_threshold' : -1
}

dual_mom_parameters = {
    'dm_tickers_list' : ['VNQ', 'EEM', 'DBC', 'SPY', 'TLT', 'SHY'],
    'dm_rebalance_unit' : 'Month',
    'dm_rebalance_freq' : 1,
    'dm_rebalance_shift' : 0,
    'dm_lb_period' : 4,
    'dm_skipped_period' : 0,
    'dm_no_played_ETFs' : 3,
    'dm_sub_rank_weights' : {'relMom' : 0.5, 'volatility' : 0.25, 'correlation' : 0.25},
    'dm_abs_threshold' : 0
}

pv_played_fin, strat_played_rets, strat_played_weights, pos_played_fin, cash_played_fin, strat_played_curr_weights, strat_unlev_curr_weights, ETF_curr_levs, monthly_curr_lev, tlt_curr_lev = leveraged_ew_meta(meta_parameters, taa_parameters, bold_parameters, dual_mom_parameters, meta_leverage_parameters)
pd.set_option('display.float_format', '{:.2f}%'.format)
print('Unleveraged weights: ')
display(strat_unlev_curr_weights * 100)
print('Leveraged weights: ')
display(strat_played_curr_weights * 100)
print('Used overall leverage: ', '{:.2f}%'.format(meta_leverage_parameters['meta_overall_leverage'] * 100))
print('Used TLT based leverage: ', '{:.2f}%'.format(tlt_curr_lev * 100))
print('Used monthly performance based leverage: ', '{:.2f}%'.format(monthly_curr_lev * 100))
print('Used ETF performance based leverage: ')
display(ETF_curr_levs * 100)
pf.create_simple_tear_sheet(strat_played_rets)
